##  Crash Data Request on Hutchison River Parkway 

#### Raw data: Crashes and associated records from the vehicle and contributing factors tables (2013-2017)

#### Database: DOTDEVRHPGSQL01
#### Tables: archive."2019_11_13_nysdot_all", archive."2019_11_13_nysdot_vehicle"



In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2019-12-10_18:57
E:\RIS\Staff Folders\Samuel\Requests\SH\Hutchison River Parkway


In [2]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-10 18:57:40


In [3]:
cdb.csv_to_table(input_file='E:\RIS\Staff Folders\Samuel\Requests\SH\Hutchison River Parkway\CrashRequest_RefMarker.csv',schema='working', table_name = 'hriver', overwrite=True)

Reading data into Database



42it [00:01, 39.07it/s]



42 rows added to working.hriver



### nysdot_all HRiver Data

In [4]:
nysdot_all = cdb.dfquery("""

SELECT * FROM archive."2019_11_13_nysdot_all" nysdot_a
JOIN working.hriver 
    ON nysdot_a.ref_mrkr = working.hriver.panel
WHERE case_yr BETWEEN 2013 and 2017

""")  

In [5]:
nysdot_all.to_csv('nysdot_all_hriver.csv',index=False)

### nysdot_vehicle HRiver Data

In [6]:
nysdot_vehicle = cdb.dfquery("""    

WITH DATA AS

--Selects all the crashids from nysdot_all joined with the hriver table on ref_mrkr 
(SELECT DISTINCT crashid FROM archive."2019_11_13_nysdot_all" nysdot_a
JOIN working.hriver 
ON nysdot_a.ref_mrkr = working.hriver.panel
WHERE case_yr BETWEEN 2013 AND 2017)


SELECT * FROM  archive."2019_11_13_nysdot_vehicle" nysdot_v
WHERE crashid IN (SELECT * FROM data) 

""")
                             

In [7]:
nysdot_vehicle.to_csv('nysdot_vehicle_hriver.csv',index=False)

### nysdot_appfactor HRiver Data

In [8]:
nysdot_appfactor = cdb.dfquery("""

WITH DATA AS

--Selects all the crashids from nysdot_all joined with the hriver table on ref_mrkr
(SELECT DISTINCT crashid FROM archive."2019_11_13_nysdot_all" nysdot_a
JOIN working.hriver 
ON nysdot_a.ref_mrkr = working.hriver.panel
WHERE case_yr BETWEEN 2013 AND 2017)


SELECT * FROM  archive."2019_11_13_nysdot_appfactor" nysdot_app
WHERE crashid IN (SELECT * FROM data)

""")

In [9]:
nysdot_appfactor.to_csv('nysdot_appfactor_hriver.csv',index=False)